# MPh (mph) — detailed tutorial and reference

This notebook is a thorough, **local-source** tutorial and reference for the 
`mph` package (Pythonic scripting interface for COMSOL Multiphysics). It is 
based on the installed package source in `./.venv/Lib/site-packages/mph/` 
(version 1.3.1).

**Audience**: Users who want to automate COMSOL models with Python and need 
both a learning path and a reference for the public API.

**Important constraints**
- COMSOL must be installed and licensed.
- `mph` uses JPype to bridge to COMSOL’s Java API. Only **one client per Python 
  process** is allowed. For parallel runs, use multiple Python processes.
- Many operations are no-ops unless COMSOL is available.


## Contents
1. Quick start
2. Configuration (`mph.config`)
3. Session control (`mph.start`)
4. Client (`mph.Client`)
5. Server (`mph.Server`)
6. Model (`mph.Model`)
7. Node (`mph.Node`)
8. Tree and inspect helpers (`mph.tree`, `mph.inspect`)
9. Common workflows (end‑to‑end patterns)
10. Troubleshooting & performance notes


## 1. Quick start

This section shows a minimal end‑to‑end flow: start COMSOL, load a model, 
change parameters, solve, evaluate, export, and save.


In [ ]:
import mph

# Configure before starting
mph.option('session', 'client-server')  # or 'stand-alone' on Windows
mph.option('caching', True)

client = mph.start(cores=4)

# Load a model (replace with your file)
model = client.load('model.mph')

# Change parameters
model.parameter('U', '5[V]')
model.parameter('d', '1[mm]')

# Solve
model.build()
model.mesh()
model.solve()

# Evaluate
C = model.evaluate('2*es.intWe/U^2', unit='pF')
print('capacitance:', C)

# Export (if export node exists)
# model.export('plot 1', 'result.png')

# Save
model.save('model_updated.mph')

# Cleanup
client.remove(model)


## 2. Configuration (`mph.config`)

`mph.option()` accesses and modifies configuration options. The built‑in 
options are: 
- `session`: `'client-server'` or `'stand-alone'`
- `caching`: `True`/`False` (reuse models loaded from the same path)
- `classkit`: `True`/`False` (enable COMSOL classkit)

Configuration is loaded from `MPh.ini` if found. The search order is:
1. Current working directory
2. User config directory (platform dependent)
3. Package folder

You can explicitly save a config file using `mph.config.save()`.


In [ ]:
import mph
from mph import config

# Read all options
print(mph.option(None))

# Set individual options
mph.option('session', 'client-server')
mph.option('caching', True)
mph.option('classkit', False)

# Save config to default location
# config.save()

# Load config from a specific file
# config.load('MPh.ini')


## 3. Session control (`mph.start`)

`mph.start()` is the convenience entry point for most users. It returns a 
`Client` instance and manages the JVM lifecycle.

Key details from the implementation:
- Enforces **one client per Python process** (COMSOL API singleton).
- Uses `session` option to decide client‑server vs stand‑alone.
- On Windows, stand‑alone is allowed; otherwise client‑server is default.
- Automatically starts and stops the JVM.


In [ ]:
import mph

client = mph.start(cores=2, version=None, port=0)
print('cores:', client.cores)


## 4. Client (`mph.Client`)

A `Client` wraps COMSOL’s `ModelUtil` Java singleton. It starts the JVM, 
connects to a server (optional), and owns the loaded models.

### Key methods and properties
- `cores`: number of CPU cores in use
- `models()`, `names()`, `files()`: inspect loaded models
- `modules()`: list licensed COMSOL modules
- `load(file)`: load a `.mph` file
- `create(name=None)`: create a new empty model
- `remove(model)`: remove a model by name or object
- `clear()`: remove all models
- `caching(True/False)`: enable or disable path-based model caching
- `connect(port, host='localhost')`: connect to a COMSOL server
- `disconnect()`: disconnect from server


In [ ]:
# Example client usage
client = mph.start(cores=2)

print('licensed modules:', client.modules())
print('loaded model names:', client.names())

# Load (with caching enabled)
client.caching(True)
model = client.load('model.mph')

# Access by name using division operator
# same_model = client/'Model 1'

# Remove
# client.remove(model)


## 5. Server (`mph.Server`)

The `Server` class starts a COMSOL server process and exposes its port. 
Use it when you want explicit control over server lifecycle or ports.

### Key arguments
- `cores`: limit CPU cores
- `version`: choose specific COMSOL version
- `port`: integer, `0` = random free port
- `multi`: `'on'` or `'off'` to keep server alive for multiple clients
- `timeout`: server startup timeout
- `arguments`: extra CLI args passed to COMSOL server


In [ ]:
# Explicit server usage
# server = mph.Server(cores=2, port=0, multi='on')
# print('server port:', server.port)
# client = mph.Client(cores=2, port=server.port)
# ... work ...
# client.disconnect()
# server.stop()


## 6. Model (`mph.Model`)

A `Model` is a wrapper around the COMSOL `com.comsol.model.Model` object. 
It focuses on loading, inspecting, solving, and evaluating models.

### Inspection
- `name()`, `file()`, `version()`
- `functions()`, `components()`, `geometries()`, `selections()`
- `physics()`, `multiphysics()`, `materials()`
- `meshes()`, `studies()`, `solutions()`
- `datasets()`, `plots()`, `exports()`
- `modules()`: required modules/products
- `problems()`: aggregate warnings/errors from the model tree

### Solving
- `build(geometry=None)`
- `mesh(mesh=None)`
- `solve(study=None)`

### Evaluation
- `inner(dataset=None)`: inner solution indices/values (time‑dependent)
- `outer(dataset=None)`: outer sweep indices/values (parametric sweeps)
- `evaluate(expr, unit=None, dataset=None, inner=None, outer=None)`

### Parameters
- `parameter(name, value=None, evaluate=False)`
- `parameters(evaluate=False)`
- `description(name, text=None)` / `descriptions()`

### Properties & features
- `property(node, name, value=None)`
- `properties(node)`
- `create(node, *args)`
- `remove(node)`

### Files
- `import_(node, file)`
- `export(node=None, file=None)`
- `clear()` (clear solution/mesh/plot data)
- `reset()` (reset modeling history)
- `save(path=None, format=None)`


In [ ]:
# Example: model inspection and solve
model = client.load('model.mph')
print(model.name(), model.file(), model.version())
print('physics:', model.physics())
print('studies:', model.studies())

# Build/mesh/solve
model.build()
model.mesh()
model.solve()


In [ ]:
# Example: evaluate results
inner_idx, inner_vals = model.inner()
outer_idx, outer_vals = model.outer()
print('inner:', inner_idx)
print('outer:', outer_idx)

value = model.evaluate('V')  # uses default dataset
print('V:', value)


In [ ]:
# Example: parameters
print(model.parameters())
model.parameter('U', '10[V]')
print('U (eval):', model.parameter('U', evaluate=True))


## 7. Node (`mph.Node`)

`Node` is the core navigation and manipulation type. Use `/` to navigate.

### Navigation and inspection
- `name()`, `tag()`, `type()`
- `parent()`, `children()`, `exists()`
- `is_root()`, `is_group()`
- `comment(text=None)`
- `problems()` (warnings/errors under the node)

### Interaction
- `rename(name)` / `retag(tag)`
- `property(name, value=None)` / `properties()`
- `select(entity)` / `selection()`
- `toggle(action)` (enable/disable)
- `run()`
- `import_(file)`
- `create(*args, name=None)`
- `remove()`

### Selection details
`select(entity)` accepts:
- a selection node (named selection)
- a list/array of integers (manual selection)
- `'all'` to select everything
- `None` to clear

### Feature creation
`create()` works for COMSOL feature nodes (not all nodes). Use feature 
types shown in the COMSOL GUI Settings tab (e.g., `'Block'`, `'Sphere'`).


In [ ]:
# Example: Node navigation
root = model/None
print('root children:', [n.name() for n in root.children()])

physics = model/'physics'
for child in physics.children():
    print(child.name(), child.type())


In [ ]:
# Example: properties and selection
node = model/'physics/ht'  # example path, adjust to your model
if node.exists():
    print(node.properties())

# Example: selection (manual)
# node.select([1, 2, 3])
# print(node.selection())


## 8. Tree and inspect helpers

`mph.tree(node, max_depth=None)` prints a text tree of the model.
`mph.inspect(java_or_node)` prints a Java object’s methods and properties.

Note: tree traversal is slow in client‑server mode. Prefer stand‑alone 
sessions for large models or limit depth.


In [ ]:
# Tree (limit depth for performance)
mph.tree(model, max_depth=2)

# Inspect Java object
mph.inspect((model/'studies').java)


## 9. Common workflows

### A) Parameter sweep (manual)
Loop over parameter values, solve, and extract a scalar.


In [ ]:
values = [1, 2, 5, 10]
results = []
for v in values:
    model.parameter('U', f'{v}[V]')
    model.solve()
    C = model.evaluate('2*es.intWe/U^2', unit='pF')
    results.append((v, C))

print(results)


### B) Export a plot or dataset
Use export nodes configured in the model or override filenames.


In [ ]:
# Export a specific node
# model.export('plot 1', 'plot.png')

# Export all export nodes with their configured filenames
# model.export()


### C) Inspect model problems
Use `model.problems()` to find warnings or errors across the model.


In [ ]:
problems = model.problems()
for p in problems:
    print(p['category'], p['message'], p['node'])


## 10. Troubleshooting & performance notes

- **One client per process**: the COMSOL API singleton enforces this.
- **Client‑server overhead**: tree traversal and deep node operations can be slow.
- **Stand‑alone mode**: faster; typically only supported on Windows.
- **Caching**: enable `client.caching(True)` to avoid reloading the same file.
- **Complex results**: `evaluate()` returns complex arrays when available.
- **Node selection**: geometry nodes require Java-level selection APIs.
